In [25]:
import pandas as pd
from sklearn.cluster import DBSCAN

from src.settings import DATA_PATH, MODELS_PATH
from src.stages.cluster import EmbeddingGrouper, FrequencyVectorizer, Tokenizer, RuWikiRuscorporaCBOW_300_10_2021, \
    KeywordGroupFactory
from src.replacer import Replacer
from src.utils import keywords_from_dataframe

In [19]:
%load_ext autoreload
%autoreload 2

In [12]:
keyword_df = pd.read_excel(DATA_PATH / 'raw' / 'data_final_07_04.xlsx', 'Sheet1')

In [13]:
keyword_df.head(10)

,filename,keywords,keywords_processed,keywords_processed_len
0,(771591) РПД Основы токсикологии 19.03.01 2018...,экспериментальное определение параметров токси...,экспериментальное определение параметров токси...,3
1,(571604) РПП НИР.pdf.tables.json,"дискуссии,поиска,тч помощью цифровых технологи...",алгоритмы стратегического планирования деятель...,3
2,(324333) 41 Теория информации.pdf.tables.json,"оптимальное кодирование источников информации,...","оптимальное кодирование источников информации,...",4
3,(285588) Б1_54_Метрология_стандартизация_и_сер...,"сертификации,преобразователи,стандартизация,ос...","поиск нормативных документов, автоматизированн...",4
4,(324344) 51 Системы защиты информации в ведущи...,современное состояние международная информацио...,"международная информационная безопасность, ин...",4
5,(747520) РПД Электроника лазерных систем и ком...,модуляторы для управления электрооптическими з...,"электрооптические затворы, накачка твердотельн...",4
6,(324345) 51 Проектирование комплексных систем ...,"secret,net,системы защиты информации,сущность ...","системы защиты информации, принципы комплексно...",4
7,(449608) РПД НИР.pdf.tables.json,"способы,формирования градостроительных решений...","формирование градостроительных решений, технол...",4
8,(285573) Б1_49_Системы_ввода-вывода.pdf.tables...,инструментального программного аппаратного обе...,"аппаратные интерфейсы вычислительных систем, к...",4
9,(545181) Спец.2 РПД Производственная практика ...,методов организационноэкономического моделиров...,"моделирование менеджмента качества, прогнозиро...",4


In [39]:
grouper = EmbeddingGrouper(
    vectorizer=FrequencyVectorizer(
        tokenizer=Tokenizer(
            vectors=RuWikiRuscorporaCBOW_300_10_2021
                .load(MODELS_PATH / 'embeddings')
        )
    ),
    model=DBSCAN(**(
        model_params := dict(
            eps=0.08,
            min_samples=2,
            metric='cosine',
        )
    )),
    keyword_group_factory=KeywordGroupFactory(
        algorithm='most_common_bigram'  # как будет вычисляться центр группы
    ),
    verbose=True
)

c:\users\uiqko\pyenvs\mint\lib\site-packages\gensim\models\keyedvectors.py:478: UserWarning: Adding single vectors to a KeyedVectors which grows by one each time can be costly. Consider adding in batches or preallocating to the required size.
  warnings.warn(


In [26]:
keywords = keywords_from_dataframe(keyword_df.keywords_processed)

In [28]:
keywords[:10]

[['экспериментальное определение параметров токсикометрии',
  'основы токсикологии',
  'влияние организмов на среду обитания'],
 ['алгоритмы стратегического планирования деятельности организации',
  'поиск источников научноисследовательской информации',
  'аргументированное изложение собственной точки зрения'],
 ['оптимальное кодирование источников информации',
  'базовые понятия информатики вычислительной техники',
  'оптимальное кодирование',
  'дискретные источники информации'],
 ['поиск нормативных документов',
  'автоматизированные методы обработки измерительной информации',
  'региональный и локальный уровнм',
  'система стандартизации'],
 ['международная информационная безопасность',
  'информационное противодействие терроризму и кибертерроризму в открытых сетях',
  'обеспечение национальной и международной кибербезопасности',
  'социальноэкономические и культурные особенности зарубежных стран'],
 ['электрооптические затворы',
  'накачка твердотельных лазеров',
  'силовые элемен

In [40]:
replacer = Replacer(grouper.apply(keywords))

Embeddings: 100%|██████████| 12671/12671 [00:08<00:00, 1572.33it/s]


In [43]:
len(replacer.groups)

9519

In [42]:
replacer.groups[0]

KeywordGroup(center='защиты информации', keywords={'обеспечение программной защиты', 'способы средства технической защиты информации', 'защита информации в системах документооборота', 'средства технической защиты информации', 'программноаппаратные криптографические средства защиты информации', 'методы обеспечения программной защиты', 'технологии проектирования систем защиты информации', 'внедрение систем защиты информации', 'защита информации от нсд', 'контроль над системой защиты информации', 'существующие системы защиты информации мультиагентных систем', 'аппаратные средства защиты', 'системы защиты информации', 'проектирование систем защиты информации', 'построение систем защиты информации', 'защита информации', 'анализ программноаппаратных средств защиты информации', 'системы инженернотехнической защиты информации', 'криптографическая защита информации', 'тестирование системы защиты информации', 'средства защиты программного обеспечения', 'реализации программноаппаратных средств за

In [50]:
for i in range(10):
    old = set(keywords[i])
    new = set(next(replacer.replace([keywords[i]])))
    print(old.difference(new))
    print('->', new.difference(old))
    print()

{'влияние организмов на среду обитания', 'экспериментальное определение параметров токсикометрии'}
-> {'влияние организмов', 'экспериментальное определение'}

{'алгоритмы стратегического планирования деятельности организации', 'аргументированное изложение собственной точки зрения', 'поиск источников научноисследовательской информации'}
-> {'аргументированное изложение', 'поиск источников', 'алгоритмы стратегического'}

{'дискретные источники информации', 'оптимальное кодирование источников информации', 'базовые понятия информатики вычислительной техники'}
-> {'источники информации', 'базовые понятия'}

{'автоматизированные методы обработки измерительной информации', 'региональный и локальный уровнм', 'поиск нормативных документов'}
-> {'поиск нормативных', 'измерительной информации', 'региональный и'}

{'информационное противодействие терроризму и кибертерроризму в открытых сетях', 'обеспечение национальной и международной кибербезопасности', 'международная информационная безопасность'

In [51]:
keyword_df['result_keywords'] = list(map(', '.join, replacer.replace(keywords)))

In [54]:
keyword_df[['keywords_processed', 'result_keywords']].head(10)

,keywords_processed,result_keywords
0,экспериментальное определение параметров токси...,"влияние организмов, основы токсикологии, экспе..."
1,алгоритмы стратегического планирования деятель...,"аргументированное изложение, поиск источников,..."
2,"оптимальное кодирование источников информации,...","оптимальное кодирование, базовые понятия, исто..."
3,"поиск нормативных документов, автоматизированн...","поиск нормативных, система стандартизации, изм..."
4,"международная информационная безопасность, ин...","международная информационная, обеспечение наци..."
5,"электрооптические затворы, накачка твердотельн...","титансапфировый лазер, электрооптические затво..."
6,"системы защиты информации, принципы комплексно...","информационной безопасности, защиты информации..."
7,"формирование градостроительных решений, технол...","градостроительных решений, разработки градостр..."
8,"аппаратные интерфейсы вычислительных систем, к...","принципы организации, выделение вычислительног..."
9,"моделирование менеджмента качества, прогнозиро...","моделирование менеджмента, анализ экономически..."


In [56]:
keyword_df.to_csv(DATA_PATH / 'results' / 'replaced.v1.0.csv', index=False)